In [ ]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

GPU Available:  []


## Downloading the data



Follow these steps to download and prepare the data:
1. Download the train dataset file from https://www.kaggle.com/cdawn1/messy-vs-clean-room?
2. Upload it to your Colab environment. After you upload you should be able to see the file by running the `ls` command.
3. Unzip it (use the code below)

In [ ]:
FILEID='17BB2Ufj-9rTnT9cwZR8fu_sKKGdCXLxM'
FILENAME='train.zip'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILEID" -O $FILENAME && rm -rf /tmp/cookies.txt

--2020-07-22 20:21:19--  https://docs.google.com/uc?export=download&confirm=UCvL&id=17BB2Ufj-9rTnT9cwZR8fu_sKKGdCXLxM
Resolving docs.google.com (docs.google.com)... 64.233.189.138, 64.233.189.101, 64.233.189.100, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-bg-docs.googleusercontent.com/docs/securesc/68q9kulb1te4n34vpnj25rhst6evrn33/aeqppk3unqddhtg2sic62rps6dcgol5q/1595449275000/12308918870841825745/10415983267238619698Z/17BB2Ufj-9rTnT9cwZR8fu_sKKGdCXLxM?e=download [following]
--2020-07-22 20:21:19--  https://doc-0g-bg-docs.googleusercontent.com/docs/securesc/68q9kulb1te4n34vpnj25rhst6evrn33/aeqppk3unqddhtg2sic62rps6dcgol5q/1595449275000/12308918870841825745/10415983267238619698Z/17BB2Ufj-9rTnT9cwZR8fu_sKKGdCXLxM?e=download
Resolving doc-0g-bg-docs.googleusercontent.com (doc-0g-bg-docs.googleusercontent.com)... 64.233.189.132, 2404:6800:4008:c07::84
Connecting

In [ ]:
ls -lh

total 33M
drwxr-xr-x 1 root root 4.0K Jul 10 16:29 sample_data/
-rw-r--r-- 1 root root  33M Jul 22 20:21 train.zip


In [ ]:
!unzip -q train.zip -d kaggle_original_data

In [ ]:
!rm -r kaggle_original_data/__MACOSX/

In [ ]:
!ls -l kaggle_original_data | head

total 8
drwxr-xr-x 2 root root 4096 Jul 22 20:17 clean
drwxr-xr-x 2 root root 4096 Jul 22 20:17 messy


In [ ]:
!ls -l kaggle_original_data/*.jpg | wc -l

25000


In [ ]:
for target_dir in [train_dir, validation_dir, test_dir]:
    print(target_dir)

/content/messy_vs_clean_room/train
/content/messy_vs_clean_room/validation
/content/messy_vs_clean_room/test


In [ ]:
!rm -rf messy_vs_clean_room/

In [ ]:
from random import shuffle
import os, shutil

# list all labels
label_dirs = ['clean', 'messy']

# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = '/content/kaggle_original_data'

# The directory where we will
# store our smaller dataset
base_dir = '/content/messy_vs_clean_room'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training/validation/test label pictures
for target_dir in [train_dir, validation_dir, test_dir]:
    for label in label_dirs:
        dir = os.path.join(target_dir, label)
        os.mkdir(dir)

# Copy 70% of each label to train, 15% to valid, and 15% to test directories
for label in label_dirs:
    fnames = os.listdir(os.path.join(original_dataset_dir, label))
    shuffle(fnames)  # shuffling the list
    n_img_start_valid = int(len(fnames)*0.7)
    n_img_start_test  = int(len(fnames)*0.85)
    for fname in fnames[:n_img_start_valid]:  # train
        src = os.path.join(original_dataset_dir, label, fname)
        dst = os.path.join(train_dir, label, fname)
        shutil.copyfile(src, dst)
    for fname in fnames[n_img_start_valid:n_img_start_test]:  # valid
        src = os.path.join(original_dataset_dir, label, fname)
        dst = os.path.join(validation_dir, label, fname)
        shutil.copyfile(src, dst)
    for fname in fnames[n_img_start_test:]:  # test
        src = os.path.join(original_dataset_dir, label, fname)
        dst = os.path.join(test_dir, label, fname)
        shutil.copyfile(src, dst)

As a sanity check, let's count how many pictures we have in each training split (train / validation / test):

In [ ]:
total_train_imgs = 0
total_valid_imgs = 0
for label in label_dirs:
    print('total images for label', label, 'in training:', len(os.listdir(os.path.join(train_dir, label))),
                                           'in valid:', len(os.listdir(os.path.join(validation_dir, label))),
                                            'in test:', len(os.listdir(os.path.join(test_dir, label))))
    total_train_imgs += len(os.listdir(os.path.join(train_dir, label)))
    total_valid_imgs += len(os.listdir(os.path.join(validation_dir, label)))

total images for label clean in training: 74 in valid: 16 in test: 16
total images for label messy in training: 74 in valid: 16 in test: 16


In [ ]:
print('Total number of training images:', total_train_imgs)
print('Total number of validation images:', total_valid_imgs)

Total number of training images: 148
Total number of validation images: 32


## Building our network

We've already built a small convnet for MNIST in the previous example, so you should be familiar with them. We will reuse the same 
general structure: our convnet will be a stack of alternated `Conv2D` (with `relu` activation) and `MaxPooling2D` layers.

However, since we are dealing with bigger images and a more complex problem, we will make our network accordingly larger: it will have one 
more `Conv2D` + `MaxPooling2D` stage. This serves both to augment the capacity of the network, and to further reduce the size of the 
feature maps, so that they aren't overly large when we reach the `Flatten` layer. Here, since we start from inputs of size 150x150 (a 
somewhat arbitrary choice), we end up with feature maps of size 7x7 right before the `Flatten` layer.

Note that the depth of the feature maps is progressively increasing in the network (from 32 to 128), while the size of the feature maps is 
decreasing (from 148x148 to 7x7). This is a pattern that you will see in almost all convnets.

Since we are attacking a binary classification problem, we are ending the network with a single unit (a `Dense` layer of size 1) and a 
`sigmoid` activation. This unit will encode the probability that the network is looking at one class or the other.